In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, cohen_kappa_score
from sklearn.preprocessing import MinMaxScaler

In [3]:
df = pd.read_csv('data.csv', index_col=0)

In [4]:
df

,full_text,Overall,Cohesion,Syntax,Vocabulary,Phraseology,Grammar,Conventions,Overall_normalized,Cohesion_normalized,Syntax_normalized,Vocabulary_normalized,Phraseology_normalized,Grammar_normalized,Conventions_normalized
0,Imagine if you could prove other people that y...,4.0,3.5,4.0,3.5,3.5,4.0,4.0,0.750,0.625,0.750,0.625,0.625,0.750,0.750
1,I think that students would benefit from learn...,3.5,3.5,3.5,3.0,3.0,4.0,3.0,0.625,0.625,0.625,0.500,0.500,0.750,0.500
2,When a problem is a change you have to let it ...,2.5,2.5,2.5,3.0,2.0,2.0,2.5,0.375,0.375,0.375,0.500,0.250,0.250,0.375
3,"Dear, Principal\n\nIf u change the school poli...",3.0,3.0,3.5,3.0,3.0,3.0,2.5,0.500,0.500,0.625,0.500,0.500,0.500,0.375
4,The best time in life is when you become yours...,4.5,4.5,4.5,4.5,4.5,4.0,5.0,0.875,0.875,0.875,0.875,0.875,0.750,1.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6477,Some schools offer distance learning as a opti...,3.0,2.5,2.0,3.0,3.5,3.0,3.0,0.500,0.375,0.250,0.500,0.625,0.500,0.500
6478,I disagree that attend classes from home can b...,3.0,3.0,3.0,2.5,3.5,3.5,3.0,0.500,0.500,0.500,0.375,0.625,0.625,0.500
6479,Introduction in my opinion kids should not sta...,3.0,3.0,3.0,4.0,3.0,3.5,3.0,0.500,0.500,0.500,0.750,0.500,0.625,0.500
6480,"some times, We don't succed in life for reason...",3.0,3.5,3.5,3.0,3.0,3.0,2.5,0.500,0.625,0.625,0.500,0.500,0.500,0.375


In [5]:
scores = ['Overall', 'Cohesion', 'Syntax', 'Vocabulary', 'Phraseology', 'Grammar', 'Conventions']
scores_normalized = ['Overall_normalized', 'Cohesion_normalized', 'Syntax_normalized', 'Vocabulary_normalized', 'Phraseology_normalized', 'Grammar_normalized', 'Conventions_normalized']

In [6]:
y = df[scores]

In [7]:
y_norm = df[scores_normalized]

In [ ]:
!pip install catboost

In [11]:
X = df.drop(scores, axis=1)
X = X.drop(scores_normalized, axis=1)

In [14]:
from catboost import CatBoostRegressor, Pool

In [19]:
for aspect in scores_normalized:
    X_train, X_test, y_train, y_test = train_test_split(X[['full_text']], y_norm[aspect], test_size=0.2, random_state=42)

    train_pool = Pool(X_train, y_train, text_features=['full_text'])
    test_pool = Pool(X_test, text_features=['full_text'])

    model = CatBoostRegressor(iterations=500, depth=6, learning_rate=0.1, verbose=50)
    model.fit(train_pool)

    y_pred = model.predict(test_pool)

    print(aspect)
    print("Mean Squared Error:", mean_squared_error(y_test, y_pred))
    print("Mean Absolute Error:", mean_absolute_error(y_test, y_pred))
    print("R² Score:", r2_score(y_test, y_pred))

    y_pred = y_pred * 4 + 1 # to original values
    y_pred = np.round(y_pred * 2) / 2
    y_test = y_test * 4 + 1
    y_test = np.round(y_test * 2) / 2

    y_pred = (y_pred - 1) * 2 # to integer values
    y_test = (y_test - 1) * 2

    print("QWK Score:", cohen_kappa_score(y_test, y_pred), '\n')

0:	learn: 0.1576573	total: 344ms	remaining: 2m 51s
50:	learn: 0.1213529	total: 12.9s	remaining: 1m 53s
100:	learn: 0.1108174	total: 25.4s	remaining: 1m 40s
150:	learn: 0.1015309	total: 38.1s	remaining: 1m 27s
200:	learn: 0.0940187	total: 50.6s	remaining: 1m 15s
250:	learn: 0.0875234	total: 1m 2s	remaining: 1m 2s
300:	learn: 0.0815917	total: 1m 15s	remaining: 49.9s
350:	learn: 0.0762858	total: 1m 27s	remaining: 37.2s
400:	learn: 0.0717101	total: 1m 40s	remaining: 24.7s
450:	learn: 0.0673705	total: 1m 52s	remaining: 12.2s
499:	learn: 0.0636810	total: 2m 5s	remaining: 0us
Overall_normalized
Mean Squared Error: 0.014666296876818011
Mean Absolute Error: 0.0971905770379441
R² Score: 0.37898657497233956
QWK Score: 0.15878036160645959 

0:	learn: 0.1623463	total: 416ms	remaining: 3m 27s
50:	learn: 0.1320335	total: 12.6s	remaining: 1m 50s
100:	learn: 0.1221907	total: 24.7s	remaining: 1m 37s
150:	learn: 0.1122120	total: 37.2s	remaining: 1m 25s
200:	learn: 0.1041913	total: 49.8s	remaining: 1m 14s